# DATA TRANSFORMATION - 3_ACT_DESAGREGADO
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [ ]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [10]:
modulo = 'ACT_DESAGREGADO'

In [11]:
dcc = gf.diccionario_llaves()

In [12]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,NOMBRE ENCUESTA\n,PREGUNTA\n,RESPUESTA,fecha,year,month,year_month
0,535255,Prueba de control del Asma ACT,DURANTE LAS ULTIMAS 4 SEMANAS ¿CON QUE FRECUEN...,POCAS VECES,2012-06-29,2012,6,201206
1,535255,Prueba de control del Asma ACT,DURANTE LAS ULTIMAS 4 SEMANAS ¿CON QUE FRECUEN...,1 O 2 VECES A LA SEMANA,2012-06-29,2012,6,201206
2,535255,Prueba de control del Asma ACT,DURANTE LAS ULTIMAS 4 SEMANAS ¿CON QUE FRECUEN...,DE 2 A 3 NOCHES EN A LA SEMANA,2012-06-29,2012,6,201206
3,535255,Prueba de control del Asma ACT,DURANTE LAS ULTIMAS 4 SEMANAS ¿CON QUE FRECUEN...,1 VEZ A LA SEMANA O MENOS,2012-06-29,2012,6,201206
4,535255,Prueba de control del Asma ACT,¿COMO CALIFICARIA EL CONTROL DE SU ASMA DURANT...,BIEN CONTROLADA,2012-06-29,2012,6,201206


## 2. Transform

In [16]:
base.rename(columns={ 'NOMBRE ENCUESTA\n':'nombre', 'PREGUNTA\n':'pregunta', 'RESPUESTA':'respuesta' }, inplace=True)
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602 entries, 0 to 8601
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          8602 non-null   int64         
 1   nombre      8602 non-null   object        
 2   pregunta    8602 non-null   object        
 3   respuesta   8602 non-null   object        
 4   fecha       8602 non-null   datetime64[ns]
 5   year        8602 non-null   int64         
 6   month       8602 non-null   int64         
 7   year_month  8602 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 537.8+ KB


In [17]:
# Cambiar texto a minuscula.
base = gf.letra_lower(base,'nombre')

# Elimina dplicados
base = base.drop_duplicates()

# Elimina columnas sobran
base = base.drop(columns=['nombre'])

In [19]:
base

,id,pregunta,respuesta,fecha,year,month,year_month
0,535255,durante las ultimas 4 semanas ¿con que frecuen...,pocas veces,2012-06-29,2012,6,201206
1,535255,durante las ultimas 4 semanas ¿con que frecuen...,1 o 2 veces a la semana,2012-06-29,2012,6,201206
2,535255,durante las ultimas 4 semanas ¿con que frecuen...,de 2 a 3 noches en a la semana,2012-06-29,2012,6,201206
3,535255,durante las ultimas 4 semanas ¿con que frecuen...,1 vez a la semana o menos,2012-06-29,2012,6,201206
4,535255,¿como calificaria el control de su asma durant...,bien controlada,2012-06-29,2012,6,201206
...,...,...,...,...,...,...,...
8597,744564,durante las ultimas 4 semanas ¿con que frecuen...,pocas veces,2020-05-20,2020,5,202005
8598,744564,durante las ultimas 4 semanas ¿con que frecuen...,1 o 2 veces a la semana,2020-05-20,2020,5,202005
8599,744564,durante las ultimas 4 semanas ¿con que frecuen...,nunca,2020-05-20,2020,5,202005
8600,744564,durante las ultimas 4 semanas ¿con que frecuen...,1 vez a la semana o menos,2020-05-20,2020,5,202005


Aca decidimos solo quedarnos con las 4 primeras, que son preguntas mensuales.

In [36]:
# lista de preguntas seleccionadas
lista = pd.DataFrame(base["pregunta"].value_counts()).reset_index().iloc[:5,0]

# cambiar nombre de las preguntas:
corto = np.array(['uso_inhalador','efecto_sueño','efecto_actividad usual','control_asma','falta_aire'])

In [37]:

for j in range(len(lista)):
    base = base.replace(to_replace =lista[j], 
                 value =corto[j]) 

In [38]:
# Quedandonos solo con las 5 preguntas
df_list = []
for i in corto: 
    temp_df = base[base['pregunta'] == i]
    df_list.append(temp_df)  # store for later concatenation

# si hay varios factores solo toca los distintos
base = pd.concat(df_list, axis=0).reset_index(drop=True)
base

,id,pregunta,respuesta,fecha,year,month,year_month
0,535255,uso_inhalador,de 2 a 3 noches en a la semana,2012-06-29,2012,6,201206
1,1680091,uso_inhalador,una o dos veces,2012-09-27,2012,9,201209
2,739693,uso_inhalador,una vez a la semana,2012-11-09,2012,11,201211
3,1101180,uso_inhalador,una o dos veces,2012-06-29,2012,6,201206
4,895520,uso_inhalador,mas de 4 noches a la semana,2012-09-05,2012,9,201209
...,...,...,...,...,...,...,...
8509,1266254,falta_aire,algo controlada,2020-03-12,2020,3,202003
8510,1007665,falta_aire,bien controlada,2020-04-14,2020,4,202004
8511,1331954,falta_aire,bien controlada,2020-03-19,2020,3,202003
8512,1671919,falta_aire,mal controlada,2020-04-20,2020,4,202004


In [39]:
# Conversión escala categorica a numerica
corta = [1, 2, 3, 4, 5]

falta = ['nunca',
         '1 o 2 veces a la semana',
         'de 3 a 6 veces a la semana',
         'una vez al dia',
         'mas de una vez al dia']

uso = ['nada controlada',
         'mal controlada',
         'algo controlada',
         'bien controlada',
         'totalmente controlada']

control = ['nunca',
         'una vez a la semana',
         'una o dos veces',
         'de 2 a 3 noches en a la semana',
         'mas de 4 noches a la semana']

sueño = ['nunca',
         'pocas veces',
         'algunas veces',
         'casi siempre',
         'siempre']

usual = ['nunca',
         '1 vez a la semana o menos',
         '2 o 3 veces a la semana',
         '1 o 2 veces al dia',
         'mas de 3 veces al dia']

listado = [falta, uso, control, sueño, usual]

In [40]:
df1 = base.copy()

for i in listado:
    df1 = df1.replace(i, corta)
df1

,id,pregunta,respuesta,fecha,year,month,year_month
0,535255,uso_inhalador,4,2012-06-29,2012,6,201206
1,1680091,uso_inhalador,3,2012-09-27,2012,9,201209
2,739693,uso_inhalador,2,2012-11-09,2012,11,201211
3,1101180,uso_inhalador,3,2012-06-29,2012,6,201206
4,895520,uso_inhalador,5,2012-09-05,2012,9,201209
...,...,...,...,...,...,...,...
8509,1266254,falta_aire,3,2020-03-12,2020,3,202003
8510,1007665,falta_aire,4,2020-04-14,2020,4,202004
8511,1331954,falta_aire,4,2020-03-19,2020,3,202003
8512,1671919,falta_aire,2,2020-04-20,2020,4,202004


In [41]:
# pivot table to have separate variables, nans replaced with zero
base_p = df1.pivot_table(index=['id','year','month'], columns='pregunta', values=['respuesta']).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.head()


,id_,year_,month_,respuesta_control_asma,respuesta_efecto_actividad usual,respuesta_efecto_sueño,respuesta_falta_aire,respuesta_uso_inhalador
0,502989,2014,10,1.0,1.0,1.0,4.0,1.0
1,502989,2017,1,2.0,1.0,2.0,4.0,3.0
2,502989,2018,9,1.0,1.0,1.0,5.0,1.0
3,502989,2020,4,1.0,1.0,1.0,5.0,1.0
4,506808,2015,12,5.0,2.0,3.0,3.0,5.0


In [42]:
base_p.rename( columns={ 'id_':'id', 'year_':'year', 'month_':'month' }, inplace=True)

## 3. Merge with ids table

In [44]:
# 2016 - 2020
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_actd = ids_mensual.merge( base_p, how='left')
base_final_actd.head()

,id,year,month,respuesta_control_asma,respuesta_efecto_actividad usual,respuesta_efecto_sueño,respuesta_falta_aire,respuesta_uso_inhalador
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN


In [45]:
base_final_actd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                53280 non-null  int64  
 1   year                              53280 non-null  int64  
 2   month                             53280 non-null  int64  
 3   respuesta_control_asma            1235 non-null   float64
 4   respuesta_efecto_actividad usual  1235 non-null   float64
 5   respuesta_efecto_sueño            1235 non-null   float64
 6   respuesta_falta_aire              1235 non-null   float64
 7   respuesta_uso_inhalador           1235 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 3.7 MB
